# Functions

In [1]:
import pandas as pd
import requests
from typing import Dict, List
import re
import json

def get_github_folder_files(username: str, repo_name: str, folder_path: str,
                           file_extension: str = ".csv") -> List[str]:
    """
    GitHub API를 사용해서 특정 폴더의 파일 목록을 가져오는 함수

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")
    file_extension : str
        필터링할 파일 확장자 (기본값: ".csv")

    Returns:
    --------
    List[str]
        해당 폴더의 CSV 파일명 목록
    """

    # GitHub API URL
    api_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{folder_path}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()

        files_data = response.json()

        # CSV 파일만 필터링
        csv_files = []
        for file_info in files_data:
            if file_info['type'] == 'file' and file_info['name'].endswith(file_extension):
                csv_files.append(file_info['name'])

        return sorted(csv_files)

    except requests.exceptions.RequestException as e:
        print(f"❌ GitHub API 요청 실패: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: {e}")
        return []


def create_variable_name_from_filename(filename: str) -> str:
    """
    파일명에서 변수명을 생성하는 함수
    temp1_governor_지역명_숫자.csv → g숫자_지역명

    Parameters:
    -----------
    filename : str
        파일명 (예: "temp1_governor_busan_3.csv", "temp1_governor_seoul_4.csv")

    Returns:
    --------
    str
        변수명 (예: "g3_busan", "g4_seoul")
    """

    # temp1_governor_지역명_숫자.csv → g숫자_지역명
    match = re.search(r'temp1_governor_([^_]+)_(\d+)\.csv', filename)
    if match:
        region = match.group(1)
        number = match.group(2)
        return f"g{number}_{region}"

    # 매칭되지 않으면 기본적으로 파일명 정리해서 반환
    base_name = filename.replace('.csv', '').replace('temp1_', '')
    return base_name


def load_all_csvs_from_github_folder(username: str, repo_name: str, folder_path: str) -> Dict[str, pd.DataFrame]:
    """
    GitHub 폴더에서 모든 CSV 파일을 자동으로 읽어와서 DataFrame 딕셔너리로 반환

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름 (예: "korean-elections")
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")

    Returns:
    --------
    Dict[str, pd.DataFrame]
        변수명을 키로 하고 DataFrame을 값으로 하는 딕셔너리
    """

    # 1. GitHub API로 파일 목록 가져오기
    csv_files = get_github_folder_files(username, repo_name, folder_path)

    if not csv_files:
        print("❌ CSV 파일을 찾을 수 없습니다.")
        return {}

    # 2. 각 CSV 파일 읽어오기
    base_url = f"https://raw.githubusercontent.com/{username}/{repo_name}/refs/heads/main/{folder_path}/"
    dataframes = {}

    for filename in csv_files:
        try:
            # URL 생성
            url = base_url + filename

            # 변수명 생성
            var_name = create_variable_name_from_filename(filename)

            # CSV 읽기
            df = pd.read_csv(url)

            dataframes[var_name] = df
            print(f"✅ {filename} → {var_name}")

        except Exception as e:
            print(f"❌ {filename} 읽기 실패: {e}")

    return dataframes


def create_individual_variables(dataframes: Dict[str, pd.DataFrame], globals_dict):
    """
    딕셔너리로 저장된 DataFrame들을 개별 변수로 생성

    Parameters:
    -----------
    dataframes : Dict[str, pd.DataFrame]
        load_all_csvs_from_github_folder()의 반환값
    globals_dict : dict
        변수를 저장할 전역 네임스페이스 (globals()를 전달)
    """

    for var_name, df in dataframes.items():
        globals_dict[var_name] = df
        print(f"변수 생성: {var_name}")


# # 사용 예시
# def example_usage():
#     """사용 예시"""

#     print("=== GitHub 폴더 자동 CSV 로더 ===\n")

#     print("사용법:")
#     print("""
#     username = "your-username"
#     repo_name = "korean-elections"
#     folder_path = "temp/v1_g/3rd_2002"

#     # 모든 CSV 파일을 자동으로 읽어와서 딕셔너리로 저장
#     election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

#     # 방법 1: 딕셔너리로 접근
#     busan_df = election_data['g3_busan']
#     seoul_df = election_data['g3_seoul']

#     # 방법 2: 개별 변수로 생성
#     create_individual_variables(election_data, globals())
#     # 이제 g3_busan, g3_seoul 등을 직접 사용 가능
#     """)

#     print("\n다양한 번호 지원:")
#     print("""
#     # 3회차: temp1_governor_busan_3.csv → g3_busan
#     election_3rd = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/3rd_2002")

#     # 4회차: temp1_governor_seoul_4.csv → g4_seoul
#     election_4th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/4th_2006")

#     # 5회차: temp1_governor_daegu_5.csv → g5_daegu
#     election_5th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/5th_2010")
#     """)


# if __name__ == "__main__":
#     example_usage()

# ### 간단한 사용법

# # 한 줄로 모든 CSV 읽기
# username = "your-github-username"
# repo_name = "korean-elections"
# folder_path = "temp/v1_g/3rd_2002"

# election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# # 사용하기
# busan_df = election_data['g3_busan']
# seoul_df = election_data['g3_seoul']

# # 또는 개별 변수로 생성
# create_individual_variables(election_data, globals())
# # 이제 g3_busan, g3_seoul 직접 사용 가능

In [2]:
import pandas as pd

def show_column_info(df):
    """
    데이터프레임의 열 이름과 인덱스를 확인하는 함수
    """
    print("=== 열 이름과 인덱스 ===")
    for idx, col_name in enumerate(df.columns):
        print(f"{idx}: {col_name}")
    print(f"\n총 열 개수: {len(df.columns)}")
    print(f"총 행 개수: {len(df)}")

def rename_columns_by_index(df, index_or_indices, new_name_or_names):
    """
    인덱스를 통해 열 이름을 변경하는 함수

    Parameters:
    - df: pandas DataFrame
    - index_or_indices: 단일 인덱스(int) 또는 인덱스 리스트([int, int, ...])
    - new_name_or_names: 단일 새 이름(str) 또는 새 이름 리스트([str, str, ...])

    Returns:
    - 열 이름이 변경된 DataFrame
    """
    # 단일 값을 리스트로 변환
    if isinstance(index_or_indices, int):
        indices = [index_or_indices]
    else:
        indices = index_or_indices

    if isinstance(new_name_or_names, str):
        new_names = [new_name_or_names]
    else:
        new_names = new_name_or_names

    # 길이 검증
    if len(indices) != len(new_names):
        raise ValueError(f"인덱스 개수({len(indices)})와 새 이름 개수({len(new_names)})가 일치하지 않습니다.")

    # 인덱스 유효성 검사
    for idx in indices:
        if idx < 0 or idx >= len(df.columns):
            raise ValueError(f"유효하지 않은 인덱스: {idx}. 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 열 이름 변경
    rename_dict = {}
    for idx, new_name in zip(indices, new_names):
        old_name = df.columns[idx]
        rename_dict[old_name] = new_name
        print(f"인덱스 {idx}: '{old_name}' -> '{new_name}'")

    result_df = result_df.rename(columns=rename_dict)

    print(f"\n=== 열 이름 변경 완료 ===")
    print(f"변경된 열 개수: {len(rename_dict)}")

    return result_df

def add_sum_column(df, start_idx, end_idx, new_column_name):
    """
    연속된 인덱스 범위의 열들을 합산하여 새로운 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - start_idx: 시작 인덱스
    - end_idx: 끝 인덱스 (포함)
    - new_column_name: 새로 추가할 열의 이름

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    # 유효성 검사
    if start_idx < 0 or end_idx >= len(df.columns) or start_idx > end_idx:
        raise ValueError(f"유효하지 않은 인덱스 범위: ({start_idx}, {end_idx}). 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 합산할 열 이름들 추출
    columns_to_sum = df.columns[start_idx:end_idx + 1]

    print(f"\n=== 합산할 열들 (인덱스 {start_idx}~{end_idx}) ===")
    for idx, col_name in enumerate(columns_to_sum):
        print(f"{start_idx + idx}: {col_name}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 지정된 열들을 합산하여 새로운 열 추가
    # 숫자가 아닌 값은 0으로 처리
    result_df[new_column_name] = df[columns_to_sum].select_dtypes(include=['number']).sum(axis=1, skipna=True)

    print(f"\n=== 새로운 열 추가 완료 ===")
    print(f"새로운 열 이름: {new_column_name}")
    print(f"총 열 개수: {len(result_df.columns)}")

    return result_df

def add_zero_column(df, column_name):
    """
    DataFrame 마지막에 모든 원소가 0(int)인 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - column_name: 추가할 열의 이름 (str)

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    result_df = df.copy()
    result_df[column_name] = 0
    return result_df

# # 사용 예시
# if __name__ == "__main__":
#     # CSV 파일 읽기
#     df = pd.read_csv('temp1_president_14.csv')

#     # 열 정보 확인
#     show_column_info(df)

#     # 예시 1: 단일 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 1: 단일 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df, 0, '광역시도')

#     # 예시 2: 복수 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 2: 복수 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df_renamed, [1, 2], ['시군구', '유권자수'])

#     # 예시 3: 인덱스 4~7 열들 합산 (득표수 1~4번 후보)
#     print("\n" + "="*50)
#     print("예시 3: 열 합산")
#     print("="*50)
#     result_df = add_sum_column(df_renamed, 4, 7, '상위4후보합계')

#     # 변경된 열 정보 확인
#     print("\n" + "="*50)
#     print("변경된 열 정보")
#     print("="*50)
#     show_column_info(result_df)

#     # 결과 확인 (첫 번째 행)
#     print("\n=== 첫 번째 행 결과 ===")
#     first_row = result_df.iloc[0]
#     print(f"광역시도: {first_row['광역시도']}")
#     print(f"시군구: {first_row['시군구']}")
#     print(f"유권자수: {first_row['유권자수']}")
#     print(f"득표수_1_민주자유당_김영삼: {first_row['득표수_1_민주자유당_김영삼']}")
#     print(f"득표수_2_민주당_김대중: {first_row['득표수_2_민주당_김대중']}")
#     print(f"득표수_3_통일국민당_정주영: {first_row['득표수_3_통일국민당_정주영']}")
#     print(f"득표수_4_새한국당_이종찬: {first_row['득표수_4_새한국당_이종찬']}")
#     print(f"상위4후보합계: {first_row['상위4후보합계']}")

#     # 검증: 수동 계산과 비교
#     manual_sum = (first_row['득표수_1_민주자유당_김영삼'] +
#                   first_row['득표수_2_민주당_김대중'] +
#                   first_row['득표수_3_통일국민당_정주영'] +
#                   first_row['득표수_4_새한국당_이종찬'])

#     print(f"\n수동 계산 결과: {manual_sum}")
#     print(f"함수 계산 결과: {first_row['상위4후보합계']}")
#     print(f"계산 일치: {manual_sum == first_row['상위4후보합계']}")

#     # 결과 데이터프레임 출력 (상위 5행)
#     print("\n=== 결과 데이터프레임 (상위 5행) ===")
#     print(result_df.head())

# Data (v1, 5th_2010)

In [3]:
# 한 줄로 모든 CSV 읽기
username = "sw1kwon"
repo_name = "korean-elections"
folder_path = "temp/v1_g/5th_2010"

governor_5 = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# 개별 변수로 생성
create_individual_variables(governor_5, globals())
# 이제 g5_busan, g5_seoul 직접 사용 가능

✅ temp1_governor_busan_5.csv → g5_busan
✅ temp1_governor_chungbuk_5.csv → g5_chungbuk
✅ temp1_governor_chungnam_5.csv → g5_chungnam
✅ temp1_governor_daegu_5.csv → g5_daegu
✅ temp1_governor_daejeon_5.csv → g5_daejeon
✅ temp1_governor_gangwon_5.csv → g5_gangwon
✅ temp1_governor_gwangju_5.csv → g5_gwangju
✅ temp1_governor_gyeongbuk_5.csv → g5_gyeongbuk
✅ temp1_governor_gyeonggi_5.csv → g5_gyeonggi
✅ temp1_governor_gyeongnam_5.csv → g5_gyeongnam
✅ temp1_governor_incheon_5.csv → g5_incheon
✅ temp1_governor_jeju_5.csv → g5_jeju
✅ temp1_governor_jeonbuk_5.csv → g5_jeonbuk
✅ temp1_governor_jeonnam_5.csv → g5_jeonnam
✅ temp1_governor_seoul_5.csv → g5_seoul
✅ temp1_governor_ulsan_5.csv → g5_ulsan
변수 생성: g5_busan
변수 생성: g5_chungbuk
변수 생성: g5_chungnam
변수 생성: g5_daegu
변수 생성: g5_daejeon
변수 생성: g5_gangwon
변수 생성: g5_gwangju
변수 생성: g5_gyeongbuk
변수 생성: g5_gyeonggi
변수 생성: g5_gyeongnam
변수 생성: g5_incheon
변수 생성: g5_jeju
변수 생성: g5_jeonbuk
변수 생성: g5_jeonnam
변수 생성: g5_seoul
변수 생성: g5_ulsan


## Seoul

In [4]:
g5_seoul.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_오세훈,득표수_2_민주당_한명숙,득표수_3_자유선진당_지상욱,득표수_7_진보신당_노회찬,득표수_8_미래연합_석종현,득표수_계,무효투표수,기권수
0,서울특별시,합계,8211461,4426182,2086127,2059715,90032,143459,18339,4397672,28510,3785279
1,서울특별시,종로구,138917,77812,35476,36910,1719,2653,288,77046,766,61105
2,서울특별시,중구,109000,60763,28648,28410,1198,1618,363,60237,526,48237
3,서울특별시,용산구,198044,105005,53285,44706,2533,3160,480,104164,841,93039
4,서울특별시,성동구,250316,135277,63448,63966,2534,3891,526,134365,912,115039


In [5]:
show_column_info(g5_seoul)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_오세훈
5: 득표수_2_민주당_한명숙
6: 득표수_3_자유선진당_지상욱
7: 득표수_7_진보신당_노회찬
8: 득표수_8_미래연합_석종현
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 26


### v2.1

In [6]:
g5_seoul_21 = rename_columns_by_index(g5_seoul, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_오세훈' -> '보수정당'
인덱스 5: '득표수_2_민주당_한명숙' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [7]:
g5_seoul_21 = add_sum_column(g5_seoul_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_자유선진당_지상욱
7: 득표수_7_진보신당_노회찬
8: 득표수_8_미래연합_석종현

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [8]:
g5_seoul_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_자유선진당_지상욱,득표수_7_진보신당_노회찬,득표수_8_미래연합_석종현,득표수_계,무효투표수,기권수,그외정당
0,서울특별시,합계,8211461,4426182,2086127,2059715,90032,143459,18339,4397672,28510,3785279,251830
1,서울특별시,종로구,138917,77812,35476,36910,1719,2653,288,77046,766,61105,4660
2,서울특별시,중구,109000,60763,28648,28410,1198,1618,363,60237,526,48237,3179


In [9]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_seoul_21 = g5_seoul_21[selected_columns].copy()

g5_seoul_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,서울특별시,합계,8211461,4426182,2086127,2059715,251830,4397672,28510,3785279
1,서울특별시,종로구,138917,77812,35476,36910,4660,77046,766,61105
2,서울특별시,중구,109000,60763,28648,28410,3179,60237,526,48237


In [10]:
g5_seoul_21.to_csv("temp2_1_governor_seoul_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [11]:
g5_seoul_22 = rename_columns_by_index(g5_seoul, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_오세훈' -> '보수정당'
인덱스 5: '득표수_2_민주당_한명숙' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [12]:
g5_seoul_22 = add_sum_column(g5_seoul_22, 6, 8, '그외정당')
g5_seoul_22 = add_zero_column(g5_seoul_22, '무소속')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_자유선진당_지상욱
7: 득표수_7_진보신당_노회찬
8: 득표수_8_미래연합_석종현

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [13]:
g5_seoul_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_자유선진당_지상욱,득표수_7_진보신당_노회찬,득표수_8_미래연합_석종현,득표수_계,무효투표수,기권수,그외정당,무소속
0,서울특별시,합계,8211461,4426182,2086127,2059715,90032,143459,18339,4397672,28510,3785279,251830,0
1,서울특별시,종로구,138917,77812,35476,36910,1719,2653,288,77046,766,61105,4660,0
2,서울특별시,중구,109000,60763,28648,28410,1198,1618,363,60237,526,48237,3179,0


In [14]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_seoul_22 = g5_seoul_22[selected_columns].copy()

g5_seoul_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,서울특별시,합계,8211461,4426182,2086127,2059715,251830,0,4397672,28510,3785279
1,서울특별시,종로구,138917,77812,35476,36910,4660,0,77046,766,61105
2,서울특별시,중구,109000,60763,28648,28410,3179,0,60237,526,48237


In [15]:
g5_seoul_22.to_csv("temp2_2_governor_seoul_5.csv", index=False, encoding="utf-8-sig")

## Busan

In [16]:
g5_busan.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_허남식,득표수_2_민주당_김정길,득표수_계,무효투표수,기권수
0,부산광역시,합계,2849895,1410126,770507,619565,1390072,20054,1439769
1,부산광역시,중구,41637,21350,12901,7996,20897,453,20287
2,부산광역시,서구,104507,49172,29328,19131,48459,713,55335
3,부산광역시,동구,85200,43996,25784,17380,43164,832,41204
4,부산광역시,영도구,122392,59112,30750,27332,58082,1030,63280


In [17]:
show_column_info(g5_busan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_허남식
5: 득표수_2_민주당_김정길
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 17


### v2.1 (XX)

In [18]:
g5_busan_21 = rename_columns_by_index(g5_busan, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_허남식' -> '보수정당'
인덱스 5: '득표수_2_민주당_김정길' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [19]:
g5_busan_21 = add_zero_column(g5_busan_21, '그외정당')

In [20]:
g5_busan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당
0,부산광역시,합계,2849895,1410126,770507,619565,1390072,20054,1439769,0
1,부산광역시,중구,41637,21350,12901,7996,20897,453,20287,0
2,부산광역시,서구,104507,49172,29328,19131,48459,713,55335,0


In [21]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_busan_21 = g5_busan_21[selected_columns].copy()

g5_busan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,부산광역시,합계,2849895,1410126,770507,619565,0,1390072,20054,1439769
1,부산광역시,중구,41637,21350,12901,7996,0,20897,453,20287
2,부산광역시,서구,104507,49172,29328,19131,0,48459,713,55335


In [22]:
g5_busan_21.to_csv("temp2_1_governor_busan_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [23]:
g5_busan_22 = rename_columns_by_index(g5_busan, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_허남식' -> '보수정당'
인덱스 5: '득표수_2_민주당_김정길' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [24]:
g5_busan_22 = add_zero_column(g5_busan_22, '그외정당')
g5_busan_22 = add_zero_column(g5_busan_22, '무소속')

In [25]:
g5_busan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,부산광역시,합계,2849895,1410126,770507,619565,1390072,20054,1439769,0,0
1,부산광역시,중구,41637,21350,12901,7996,20897,453,20287,0,0
2,부산광역시,서구,104507,49172,29328,19131,48459,713,55335,0,0


In [26]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_busan_22 = g5_busan_22[selected_columns].copy()

g5_busan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,부산광역시,합계,2849895,1410126,770507,619565,0,0,1390072,20054,1439769
1,부산광역시,중구,41637,21350,12901,7996,0,0,20897,453,20287
2,부산광역시,서구,104507,49172,29328,19131,0,0,48459,713,55335


In [27]:
g5_busan_22.to_csv("temp2_2_governor_busan_5.csv", index=False, encoding="utf-8-sig")

## Daegu

In [28]:
g5_daegu.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김범일,득표수_2_민주당_이승천,득표수_7_진보신당_조명래,득표수_계,무효투표수,기권수
0,대구광역시,합계,1928835,886035,633118,146458,88599,868175,17860,1042800
1,대구광역시,중구,65102,31334,22764,4867,3074,30705,629,33768
2,대구광역시,동구,266834,123904,87541,22096,11529,121166,2738,142930
3,대구광역시,서구,183845,83922,61781,12012,7938,81731,2191,99923
4,대구광역시,남구,142035,61391,45380,9059,5874,60313,1078,80644


In [29]:
show_column_info(g5_daegu)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_김범일
5: 득표수_2_민주당_이승천
6: 득표수_7_진보신당_조명래
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 9


### v2.1

In [30]:
g5_daegu_21 = rename_columns_by_index(g5_daegu, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김범일' -> '보수정당'
인덱스 5: '득표수_2_민주당_이승천' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [31]:
g5_daegu_21 = add_sum_column(g5_daegu_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_7_진보신당_조명래

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [32]:
g5_daegu_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_7_진보신당_조명래,득표수_계,무효투표수,기권수,그외정당
0,대구광역시,합계,1928835,886035,633118,146458,88599,868175,17860,1042800,88599
1,대구광역시,중구,65102,31334,22764,4867,3074,30705,629,33768,3074
2,대구광역시,동구,266834,123904,87541,22096,11529,121166,2738,142930,11529


In [33]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_daegu_21 = g5_daegu_21[selected_columns].copy()

g5_daegu_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대구광역시,합계,1928835,886035,633118,146458,88599,868175,17860,1042800
1,대구광역시,중구,65102,31334,22764,4867,3074,30705,629,33768
2,대구광역시,동구,266834,123904,87541,22096,11529,121166,2738,142930


In [34]:
g5_daegu_21.to_csv("temp2_1_governor_daegu_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [35]:
g5_daegu_22 = rename_columns_by_index(g5_daegu, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김범일' -> '보수정당'
인덱스 5: '득표수_2_민주당_이승천' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [36]:
g5_daegu_22 = add_sum_column(g5_daegu_22, 6, 6, '그외정당')
g5_daegu_22 = add_zero_column(g5_daegu_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_7_진보신당_조명래

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [37]:
g5_daegu_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_7_진보신당_조명래,득표수_계,무효투표수,기권수,그외정당,무소속
0,대구광역시,합계,1928835,886035,633118,146458,88599,868175,17860,1042800,88599,0
1,대구광역시,중구,65102,31334,22764,4867,3074,30705,629,33768,3074,0
2,대구광역시,동구,266834,123904,87541,22096,11529,121166,2738,142930,11529,0


In [38]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_daegu_22 = g5_daegu_22[selected_columns].copy()

g5_daegu_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대구광역시,합계,1928835,886035,633118,146458,88599,0,868175,17860,1042800
1,대구광역시,중구,65102,31334,22764,4867,3074,0,30705,629,33768
2,대구광역시,동구,266834,123904,87541,22096,11529,0,121166,2738,142930


In [39]:
g5_daegu_22.to_csv("temp2_2_governor_daegu_5.csv", index=False, encoding="utf-8-sig")

## Incheon

In [40]:
g5_incheon.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_안상수,득표수_2_민주당_송영길,득표수_7_진보신당_김상하,득표수_8_평화민주당_백석두,득표수_계,무효투표수,기권수
0,인천광역시,합계,2096853,1067431,469040,556902,19580,11258,1056780,10651,1029422
1,인천광역시,중구,73354,37876,17846,18216,827,458,37347,529,35478
2,인천광역시,동구,62253,35099,15714,17486,968,421,34589,510,27154
3,인천광역시,남구,337880,165592,75286,84170,2823,1721,164000,1592,172288
4,인천광역시,연수구,206407,110292,53027,53495,1978,974,109474,818,96115


In [41]:
show_column_info(g5_incheon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_안상수
5: 득표수_2_민주당_송영길
6: 득표수_7_진보신당_김상하
7: 득표수_8_평화민주당_백석두
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 11


### v2.1

In [42]:
g5_incheon_21 = rename_columns_by_index(g5_incheon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_안상수' -> '보수정당'
인덱스 5: '득표수_2_민주당_송영길' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [43]:
g5_incheon_21 = add_sum_column(g5_incheon_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_7_진보신당_김상하
7: 득표수_8_평화민주당_백석두

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [44]:
g5_incheon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_7_진보신당_김상하,득표수_8_평화민주당_백석두,득표수_계,무효투표수,기권수,그외정당
0,인천광역시,합계,2096853,1067431,469040,556902,19580,11258,1056780,10651,1029422,30838
1,인천광역시,중구,73354,37876,17846,18216,827,458,37347,529,35478,1285
2,인천광역시,동구,62253,35099,15714,17486,968,421,34589,510,27154,1389


In [45]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_incheon_21 = g5_incheon_21[selected_columns].copy()

g5_incheon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,인천광역시,합계,2096853,1067431,469040,556902,30838,1056780,10651,1029422
1,인천광역시,중구,73354,37876,17846,18216,1285,37347,529,35478
2,인천광역시,동구,62253,35099,15714,17486,1389,34589,510,27154


In [46]:
g5_incheon_21.to_csv("temp2_1_governor_incheon_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [47]:
g5_incheon_22 = rename_columns_by_index(g5_incheon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_안상수' -> '보수정당'
인덱스 5: '득표수_2_민주당_송영길' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [48]:
g5_incheon_22 = add_sum_column(g5_incheon_22, 6, 7, '그외정당')
g5_incheon_22 = add_zero_column(g5_incheon_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_7_진보신당_김상하
7: 득표수_8_평화민주당_백석두

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [49]:
g5_incheon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_7_진보신당_김상하,득표수_8_평화민주당_백석두,득표수_계,무효투표수,기권수,그외정당,무소속
0,인천광역시,합계,2096853,1067431,469040,556902,19580,11258,1056780,10651,1029422,30838,0
1,인천광역시,중구,73354,37876,17846,18216,827,458,37347,529,35478,1285,0
2,인천광역시,동구,62253,35099,15714,17486,968,421,34589,510,27154,1389,0


In [50]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_incheon_22 = g5_incheon_22[selected_columns].copy()

g5_incheon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,인천광역시,합계,2096853,1067431,469040,556902,30838,0,1056780,10651,1029422
1,인천광역시,중구,73354,37876,17846,18216,1285,0,37347,529,35478
2,인천광역시,동구,62253,35099,15714,17486,1389,0,34589,510,27154


In [51]:
g5_incheon_22.to_csv("temp2_2_governor_incheon_5.csv", index=False, encoding="utf-8-sig")

## Gwangju

In [52]:
g5_gwangju.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_정용화,득표수_2_민주당_강운태,득표수_5_민주노동당_장원섭,득표수_7_진보신당_윤난실,득표수_8_국민참여당_정찬용,득표수_9_평화민주당_조홍규,득표수_계,무효투표수,기권수
0,광주광역시,합계,1064913,529901,74490,297003,39455,30834,75830,5871,523483,6418,535012
1,광주광역시,동구,84206,44017,8096,23991,2178,2360,6247,520,43392,625,40189
2,광주광역시,서구,222260,113765,17783,61188,8643,6657,16981,1148,112400,1365,108495
3,광주광역시,남구,164726,87054,12510,53512,4336,3929,10883,847,86017,1037,77672
4,광주광역시,북구,351029,168022,22380,93471,13361,10385,25015,1663,166275,1747,183007


In [53]:
show_column_info(g5_gwangju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_정용화
5: 득표수_2_민주당_강운태
6: 득표수_5_민주노동당_장원섭
7: 득표수_7_진보신당_윤난실
8: 득표수_8_국민참여당_정찬용
9: 득표수_9_평화민주당_조홍규
10: 득표수_계
11: 무효투표수
12: 기권수

총 열 개수: 13
총 행 개수: 6


### v2.1

In [54]:
g5_gwangju_21 = rename_columns_by_index(g5_gwangju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_정용화' -> '보수정당'
인덱스 5: '득표수_2_민주당_강운태' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [55]:
g5_gwangju_21 = add_sum_column(g5_gwangju_21, 6, 9, '그외정당')


=== 합산할 열들 (인덱스 6~9) ===
6: 득표수_5_민주노동당_장원섭
7: 득표수_7_진보신당_윤난실
8: 득표수_8_국민참여당_정찬용
9: 득표수_9_평화민주당_조홍규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14


In [56]:
g5_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_5_민주노동당_장원섭,득표수_7_진보신당_윤난실,득표수_8_국민참여당_정찬용,득표수_9_평화민주당_조홍규,득표수_계,무효투표수,기권수,그외정당
0,광주광역시,합계,1064913,529901,74490,297003,39455,30834,75830,5871,523483,6418,535012,151990
1,광주광역시,동구,84206,44017,8096,23991,2178,2360,6247,520,43392,625,40189,11305
2,광주광역시,서구,222260,113765,17783,61188,8643,6657,16981,1148,112400,1365,108495,33429


In [57]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_gwangju_21 = g5_gwangju_21[selected_columns].copy()

g5_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,광주광역시,합계,1064913,529901,74490,297003,151990,523483,6418,535012
1,광주광역시,동구,84206,44017,8096,23991,11305,43392,625,40189
2,광주광역시,서구,222260,113765,17783,61188,33429,112400,1365,108495


In [58]:
g5_gwangju_21.to_csv("temp2_1_governor_gwangju_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [59]:
g5_gwangju_22 = rename_columns_by_index(g5_gwangju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_정용화' -> '보수정당'
인덱스 5: '득표수_2_민주당_강운태' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [60]:
g5_gwangju_22 = add_sum_column(g5_gwangju_22, 6, 9, '그외정당')
g5_gwangju_22 = add_zero_column(g5_gwangju_22, '무소속')


=== 합산할 열들 (인덱스 6~9) ===
6: 득표수_5_민주노동당_장원섭
7: 득표수_7_진보신당_윤난실
8: 득표수_8_국민참여당_정찬용
9: 득표수_9_평화민주당_조홍규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14


In [61]:
g5_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_5_민주노동당_장원섭,득표수_7_진보신당_윤난실,득표수_8_국민참여당_정찬용,득표수_9_평화민주당_조홍규,득표수_계,무효투표수,기권수,그외정당,무소속
0,광주광역시,합계,1064913,529901,74490,297003,39455,30834,75830,5871,523483,6418,535012,151990,0
1,광주광역시,동구,84206,44017,8096,23991,2178,2360,6247,520,43392,625,40189,11305,0
2,광주광역시,서구,222260,113765,17783,61188,8643,6657,16981,1148,112400,1365,108495,33429,0


In [62]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_gwangju_22 = g5_gwangju_22[selected_columns].copy()

g5_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,광주광역시,합계,1064913,529901,74490,297003,151990,0,523483,6418,535012
1,광주광역시,동구,84206,44017,8096,23991,11305,0,43392,625,40189
2,광주광역시,서구,222260,113765,17783,61188,33429,0,112400,1365,108495


In [63]:
g5_gwangju_22.to_csv("temp2_2_governor_gwangju_5.csv", index=False, encoding="utf-8-sig")

## Daejeon

In [64]:
g5_daejeon.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박성효,득표수_2_민주당_김원웅,득표수_3_자유선진당_염홍철,득표수_7_진보신당_김윤기,득표수_계,무효투표수,기권수
0,대전광역시,합계,1127547,596683,168616,137751,276122,9074,591563,5120,530864
1,대전광역시,동구,195072,97967,28132,21795,45415,1535,96877,1090,97105
2,대전광역시,중구,206384,109744,34128,21922,51150,1538,108738,1006,96640
3,대전광역시,서구,375073,197443,55962,44371,92799,2927,196059,1384,177630
4,대전광역시,유성구,193868,109646,28689,29079,49288,1831,108887,759,84222


In [65]:
show_column_info(g5_daejeon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_박성효
5: 득표수_2_민주당_김원웅
6: 득표수_3_자유선진당_염홍철
7: 득표수_7_진보신당_김윤기
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 6


### v2.1

In [66]:
g5_daejeon_21 = rename_columns_by_index(g5_daejeon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_박성효' -> '보수정당'
인덱스 5: '득표수_2_민주당_김원웅' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [67]:
g5_daejeon_21 = add_sum_column(g5_daejeon_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_자유선진당_염홍철
7: 득표수_7_진보신당_김윤기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [68]:
g5_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_자유선진당_염홍철,득표수_7_진보신당_김윤기,득표수_계,무효투표수,기권수,그외정당
0,대전광역시,합계,1127547,596683,168616,137751,276122,9074,591563,5120,530864,285196
1,대전광역시,동구,195072,97967,28132,21795,45415,1535,96877,1090,97105,46950
2,대전광역시,중구,206384,109744,34128,21922,51150,1538,108738,1006,96640,52688


In [69]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_daejeon_21 = g5_daejeon_21[selected_columns].copy()

g5_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대전광역시,합계,1127547,596683,168616,137751,285196,591563,5120,530864
1,대전광역시,동구,195072,97967,28132,21795,46950,96877,1090,97105
2,대전광역시,중구,206384,109744,34128,21922,52688,108738,1006,96640


In [70]:
g5_daejeon_21.to_csv("temp2_1_governor_daejeon_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [71]:
g5_daejeon_22 = rename_columns_by_index(g5_daejeon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_박성효' -> '보수정당'
인덱스 5: '득표수_2_민주당_김원웅' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [72]:
g5_daejeon_22 = add_sum_column(g5_daejeon_22, 6, 7, '그외정당')
g5_daejeon_22 = add_zero_column(g5_daejeon_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_자유선진당_염홍철
7: 득표수_7_진보신당_김윤기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [73]:
g5_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_자유선진당_염홍철,득표수_7_진보신당_김윤기,득표수_계,무효투표수,기권수,그외정당,무소속
0,대전광역시,합계,1127547,596683,168616,137751,276122,9074,591563,5120,530864,285196,0
1,대전광역시,동구,195072,97967,28132,21795,45415,1535,96877,1090,97105,46950,0
2,대전광역시,중구,206384,109744,34128,21922,51150,1538,108738,1006,96640,52688,0


In [74]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_daejeon_22 = g5_daejeon_22[selected_columns].copy()

g5_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대전광역시,합계,1127547,596683,168616,137751,285196,0,591563,5120,530864
1,대전광역시,동구,195072,97967,28132,21795,46950,0,96877,1090,97105
2,대전광역시,중구,206384,109744,34128,21922,52688,0,108738,1006,96640


In [75]:
g5_daejeon_22.to_csv("temp2_2_governor_daejeon_5.csv", index=False, encoding="utf-8-sig")

## Ulsan

In [76]:
g5_ulsan.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박맹우,득표수_5_민주노동당_김창현,득표수_7_진보신당_노옥희,득표수_계,무효투표수,기권수
0,울산광역시,합계,838805,462103,279421,133437,43256,456114,5989,376702
1,울산광역시,중구,178158,98144,63911,24441,8265,96617,1527,80014
2,울산광역시,남구,258699,137133,86398,38703,10599,135700,1433,121566
3,울산광역시,동구,132325,76495,41337,24839,9490,75666,829,55830
4,울산광역시,북구,122563,67957,35719,24584,6957,67260,697,54606


In [77]:
show_column_info(g5_ulsan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_박맹우
5: 득표수_5_민주노동당_김창현
6: 득표수_7_진보신당_노옥희
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 6


### v2.1 (XXX)

In [78]:
g5_ulsan_21 = rename_columns_by_index(g5_ulsan, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_박맹우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [79]:
g5_ulsan_21 = add_sum_column(g5_ulsan_21, 5, 6, '그외정당')
g5_ulsan_21 = add_zero_column(g5_ulsan_21, '진보정당')


=== 합산할 열들 (인덱스 5~6) ===
5: 득표수_5_민주노동당_김창현
6: 득표수_7_진보신당_노옥희

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [80]:
g5_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_5_민주노동당_김창현,득표수_7_진보신당_노옥희,득표수_계,무효투표수,기권수,그외정당,진보정당
0,울산광역시,합계,838805,462103,279421,133437,43256,456114,5989,376702,176693,0
1,울산광역시,중구,178158,98144,63911,24441,8265,96617,1527,80014,32706,0
2,울산광역시,남구,258699,137133,86398,38703,10599,135700,1433,121566,49302,0


In [81]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_ulsan_21 = g5_ulsan_21[selected_columns].copy()

g5_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,울산광역시,합계,838805,462103,279421,0,176693,456114,5989,376702
1,울산광역시,중구,178158,98144,63911,0,32706,96617,1527,80014
2,울산광역시,남구,258699,137133,86398,0,49302,135700,1433,121566


In [82]:
g5_ulsan_21.to_csv("temp2_1_governor_ulsan_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, X)

In [83]:
g5_ulsan_22 = rename_columns_by_index(g5_ulsan, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_박맹우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [84]:
g5_ulsan_22 = add_sum_column(g5_ulsan_22, 5, 6, '그외정당')
g5_ulsan_22 = add_zero_column(g5_ulsan_22, '진보정당')
g5_ulsan_22 = add_zero_column(g5_ulsan_22, '무소속')


=== 합산할 열들 (인덱스 5~6) ===
5: 득표수_5_민주노동당_김창현
6: 득표수_7_진보신당_노옥희

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [85]:
g5_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_5_민주노동당_김창현,득표수_7_진보신당_노옥희,득표수_계,무효투표수,기권수,그외정당,진보정당,무소속
0,울산광역시,합계,838805,462103,279421,133437,43256,456114,5989,376702,176693,0,0
1,울산광역시,중구,178158,98144,63911,24441,8265,96617,1527,80014,32706,0,0
2,울산광역시,남구,258699,137133,86398,38703,10599,135700,1433,121566,49302,0,0


In [86]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_ulsan_22 = g5_ulsan_22[selected_columns].copy()

g5_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,울산광역시,합계,838805,462103,279421,0,176693,0,456114,5989,376702
1,울산광역시,중구,178158,98144,63911,0,32706,0,96617,1527,80014
2,울산광역시,남구,258699,137133,86398,0,49302,0,135700,1433,121566


In [87]:
g5_ulsan_22.to_csv("temp2_2_governor_ulsan_5.csv", index=False, encoding="utf-8-sig")

## Gyeonggi

In [88]:
g5_gyeonggi.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김문수,득표수_8_국민참여당_유시민,득표수_계,무효투표수,기권수
0,경기도,합계,8761840,4534771,2271492,2079892,4351384,183387,4227069
1,경기도,수원시장안구,218041,118498,56126,58386,114512,3986,99543
2,경기도,수원시권선구,229747,113478,53993,55330,109323,4155,116269
3,경기도,수원시팔달구,174518,85692,43066,39357,82423,3269,88826
4,경기도,수원시영통구,183747,101911,46917,52842,99759,2152,81836


In [89]:
show_column_info(g5_gyeonggi)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_김문수
5: 득표수_8_국민참여당_유시민
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 45


### v2.1 (XXX)

In [90]:
g5_gyeonggi_21 = rename_columns_by_index(g5_gyeonggi, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_김문수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [91]:
g5_gyeonggi_21 = add_sum_column(g5_gyeonggi_21, 5, 5, '그외정당')
g5_gyeonggi_21 = add_zero_column(g5_gyeonggi_21, '진보정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_8_국민참여당_유시민

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 10


In [92]:
g5_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_8_국민참여당_유시민,득표수_계,무효투표수,기권수,그외정당,진보정당
0,경기도,합계,8761840,4534771,2271492,2079892,4351384,183387,4227069,2079892,0
1,경기도,수원시장안구,218041,118498,56126,58386,114512,3986,99543,58386,0
2,경기도,수원시권선구,229747,113478,53993,55330,109323,4155,116269,55330,0


In [93]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_gyeonggi_21 = g5_gyeonggi_21[selected_columns].copy()

g5_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경기도,합계,8761840,4534771,2271492,0,2079892,4351384,183387,4227069
1,경기도,수원시장안구,218041,118498,56126,0,58386,114512,3986,99543
2,경기도,수원시권선구,229747,113478,53993,0,55330,109323,4155,116269


In [94]:
g5_gyeonggi_21.to_csv("temp2_1_governor_gyeonggi_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, X)

In [95]:
g5_gyeonggi_22 = rename_columns_by_index(g5_gyeonggi, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_김문수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [96]:
g5_gyeonggi_22 = add_sum_column(g5_gyeonggi_22, 5, 5, '그외정당')
g5_gyeonggi_22 = add_zero_column(g5_gyeonggi_22, '진보정당')
g5_gyeonggi_22 = add_zero_column(g5_gyeonggi_22, '무소속')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_8_국민참여당_유시민

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 10


In [97]:
g5_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_8_국민참여당_유시민,득표수_계,무효투표수,기권수,그외정당,진보정당,무소속
0,경기도,합계,8761840,4534771,2271492,2079892,4351384,183387,4227069,2079892,0,0
1,경기도,수원시장안구,218041,118498,56126,58386,114512,3986,99543,58386,0,0
2,경기도,수원시권선구,229747,113478,53993,55330,109323,4155,116269,55330,0,0


In [98]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_gyeonggi_22 = g5_gyeonggi_22[selected_columns].copy()

g5_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경기도,합계,8761840,4534771,2271492,0,2079892,0,4351384,183387,4227069
1,경기도,수원시장안구,218041,118498,56126,0,58386,0,114512,3986,99543
2,경기도,수원시권선구,229747,113478,53993,0,55330,0,109323,4155,116269


In [99]:
g5_gyeonggi_22.to_csv("temp2_2_governor_gyeonggi_5.csv", index=False, encoding="utf-8-sig")

## Gangwon

In [100]:
g5_gangwon.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이계진,득표수_2_민주당_이광재,득표수_계,무효투표수,기권수
0,강원도,합계,1190509,741724,326111,388443,714554,27170,448785
1,강원도,춘천시,206172,124839,47183,73858,121041,3798,81333
2,강원도,원주시,234095,133502,58889,70966,129855,3647,100593
3,강원도,강릉시,170703,99246,45759,50120,95879,3367,71457
4,강원도,동해시,73611,44933,21132,22150,43282,1651,28678


In [101]:
show_column_info(g5_gangwon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_이계진
5: 득표수_2_민주당_이광재
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 19


### v2.1 (XX)

In [102]:
g5_gangwon_21 = rename_columns_by_index(g5_gangwon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_이계진' -> '보수정당'
인덱스 5: '득표수_2_민주당_이광재' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [103]:
g5_gangwon_21 = add_zero_column(g5_gangwon_21, '그외정당')

In [104]:
g5_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당
0,강원도,합계,1190509,741724,326111,388443,714554,27170,448785,0
1,강원도,춘천시,206172,124839,47183,73858,121041,3798,81333,0
2,강원도,원주시,234095,133502,58889,70966,129855,3647,100593,0


In [105]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_gangwon_21 = g5_gangwon_21[selected_columns].copy()

g5_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,강원도,합계,1190509,741724,326111,388443,0,714554,27170,448785
1,강원도,춘천시,206172,124839,47183,73858,0,121041,3798,81333
2,강원도,원주시,234095,133502,58889,70966,0,129855,3647,100593


In [106]:
g5_gangwon_21.to_csv("temp2_1_governor_gangwon_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [107]:
g5_gangwon_22 = rename_columns_by_index(g5_gangwon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_이계진' -> '보수정당'
인덱스 5: '득표수_2_민주당_이광재' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [108]:
g5_gangwon_22 = add_zero_column(g5_gangwon_22, '그외정당')
g5_gangwon_22 = add_zero_column(g5_gangwon_22, '무소속')

In [109]:
g5_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,강원도,합계,1190509,741724,326111,388443,714554,27170,448785,0,0
1,강원도,춘천시,206172,124839,47183,73858,121041,3798,81333,0,0
2,강원도,원주시,234095,133502,58889,70966,129855,3647,100593,0,0


In [110]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_gangwon_22 = g5_gangwon_22[selected_columns].copy()

g5_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,강원도,합계,1190509,741724,326111,388443,0,0,714554,27170,448785
1,강원도,춘천시,206172,124839,47183,73858,0,0,121041,3798,81333
2,강원도,원주시,234095,133502,58889,70966,0,0,129855,3647,100593


In [111]:
g5_gangwon_22.to_csv("temp2_2_governor_gangwon_5.csv", index=False, encoding="utf-8-sig")

## Chungbuk

In [112]:
g5_chungbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_정우택,득표수_2_민주당_이시종,득표수_7_진보신당_김백규,득표수_계,무효투표수,기권수
0,충청북도,합계,1183811,696393,313646,349913,19551,683110,13283,487418
1,충청북도,청주시상당구,182613,100216,46172,50754,2289,99215,1001,82397
2,충청북도,청주시흥덕구,296977,159875,69434,85301,3916,158651,1224,137102
3,충청북도,충주시,161522,94688,33714,57677,1753,93144,1544,66834
4,충청북도,제천시,107034,63341,31942,28593,1661,62196,1145,43693


In [113]:
show_column_info(g5_chungbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_정우택
5: 득표수_2_민주당_이시종
6: 득표수_7_진보신당_김백규
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 14


### v2.1

In [114]:
g5_chungbuk_21 = rename_columns_by_index(g5_chungbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_정우택' -> '보수정당'
인덱스 5: '득표수_2_민주당_이시종' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [115]:
g5_chungbuk_21 = add_sum_column(g5_chungbuk_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_7_진보신당_김백규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [116]:
g5_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_7_진보신당_김백규,득표수_계,무효투표수,기권수,그외정당
0,충청북도,합계,1183811,696393,313646,349913,19551,683110,13283,487418,19551
1,충청북도,청주시상당구,182613,100216,46172,50754,2289,99215,1001,82397,2289
2,충청북도,청주시흥덕구,296977,159875,69434,85301,3916,158651,1224,137102,3916


In [117]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_chungbuk_21 = g5_chungbuk_21[selected_columns].copy()

g5_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청북도,합계,1183811,696393,313646,349913,19551,683110,13283,487418
1,충청북도,청주시상당구,182613,100216,46172,50754,2289,99215,1001,82397
2,충청북도,청주시흥덕구,296977,159875,69434,85301,3916,158651,1224,137102


In [118]:
g5_chungbuk_21.to_csv("temp2_1_governor_chungbuk_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [119]:
g5_chungbuk_22 = rename_columns_by_index(g5_chungbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_정우택' -> '보수정당'
인덱스 5: '득표수_2_민주당_이시종' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [120]:
g5_chungbuk_22 = add_sum_column(g5_chungbuk_22, 6, 6, '그외정당')
g5_chungbuk_22 = add_zero_column(g5_chungbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_7_진보신당_김백규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [121]:
g5_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_7_진보신당_김백규,득표수_계,무효투표수,기권수,그외정당,무소속
0,충청북도,합계,1183811,696393,313646,349913,19551,683110,13283,487418,19551,0
1,충청북도,청주시상당구,182613,100216,46172,50754,2289,99215,1001,82397,2289,0
2,충청북도,청주시흥덕구,296977,159875,69434,85301,3916,158651,1224,137102,3916,0


In [122]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_chungbuk_22 = g5_chungbuk_22[selected_columns].copy()

g5_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청북도,합계,1183811,696393,313646,349913,19551,0,683110,13283,487418
1,충청북도,청주시상당구,182613,100216,46172,50754,2289,0,99215,1001,82397
2,충청북도,청주시흥덕구,296977,159875,69434,85301,3916,0,158651,1224,137102


In [123]:
g5_chungbuk_22.to_csv("temp2_2_governor_chungbuk_5.csv", index=False, encoding="utf-8-sig")

## Chungnam

In [124]:
g5_chungnam.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박해춘,득표수_2_민주당_안희정,득표수_3_자유선진당_박상돈,득표수_계,무효투표수,기권수
0,충청남도,합계,1595587,901863,154723,367288,347265,869271,32587,693724
1,충청남도,천안시서북구,221580,104685,16178,44605,42683,103466,1219,116895
2,충청남도,천안시동남구,183142,89013,14851,36201,36603,87655,1358,94129
3,충청남도,공주시,99709,61209,6888,27856,24127,58871,2338,38500
4,충청남도,보령시,85687,53693,8829,18112,24168,51109,2584,31994


In [125]:
print(154723 + 367288 + 347265)
print(154723 + 367288 + 347265 + 32587)
print(154723 + 367288 + 347265 + 32587 + 693724)
print("index 0 row 제외 선거인수 합계:", g5_chungnam.iloc[1:]['선거인수'].sum())
print("index 0 row 제외 투표수 합계:", g5_chungnam.iloc[1:]['투표수'].sum())
print("index 0 row 제외 득표수_1_한나라당_박해춘 합계:", g5_chungnam.iloc[1:]['득표수_1_한나라당_박해춘'].sum())
print("index 0 row 제외 득표수_2_민주당_안희정 합계:", g5_chungnam.iloc[1:]['득표수_2_민주당_안희정'].sum())
print("index 0 row 제외 득표수_3_자유선진당_박상돈 합계:", g5_chungnam.iloc[1:]['득표수_3_자유선진당_박상돈'].sum())
print("index 0 row 제외 득표수_계 합계:", g5_chungnam.iloc[1:]['득표수_계'].sum())
print("index 0 row 제외 무효투표수 합계:", g5_chungnam.iloc[1:]['무효투표수'].sum())
print("index 0 row 제외 기권수 합계:", g5_chungnam.iloc[1:]['기권수'].sum())

869276
901863
1595587
index 0 row 제외 선거인수 합계: 1595587
index 0 row 제외 투표수 합계: 901863
index 0 row 제외 득표수_1_한나라당_박해춘 합계: 154723
index 0 row 제외 득표수_2_민주당_안희정 합계: 367288
index 0 row 제외 득표수_3_자유선진당_박상돈 합계: 347265
index 0 row 제외 득표수_계 합계: 869271
index 0 row 제외 무효투표수 합계: 32587
index 0 row 제외 기권수 합계: 693724


In [126]:
g5_chungnam.\
assign(득표수_합=lambda df: df["득표수_1_한나라당_박해춘"] + df["득표수_2_민주당_안희정"] + df["득표수_3_자유선진당_박상돈"]).\
query("득표수_합 != 득표수_계")

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박해춘,득표수_2_민주당_안희정,득표수_3_자유선진당_박상돈,득표수_계,무효투표수,기권수,득표수_합
0,충청남도,합계,1595587,901863,154723,367288,347265,869271,32587,693724,869276
14,충청남도,홍성군,70516,43145,9686,14404,17008,41093,2047,27371,41098


In [127]:
show_column_info(g5_chungnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_박해춘
5: 득표수_2_민주당_안희정
6: 득표수_3_자유선진당_박상돈
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 18


### v2.1

In [128]:
g5_chungnam_21 = rename_columns_by_index(g5_chungnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_박해춘' -> '보수정당'
인덱스 5: '득표수_2_민주당_안희정' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [129]:
g5_chungnam_21 = add_sum_column(g5_chungnam_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_자유선진당_박상돈

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [130]:
g5_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_자유선진당_박상돈,득표수_계,무효투표수,기권수,그외정당
0,충청남도,합계,1595587,901863,154723,367288,347265,869271,32587,693724,347265
1,충청남도,천안시서북구,221580,104685,16178,44605,42683,103466,1219,116895,42683
2,충청남도,천안시동남구,183142,89013,14851,36201,36603,87655,1358,94129,36603


In [131]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_chungnam_21 = g5_chungnam_21[selected_columns].copy()

g5_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청남도,합계,1595587,901863,154723,367288,347265,869271,32587,693724
1,충청남도,천안시서북구,221580,104685,16178,44605,42683,103466,1219,116895
2,충청남도,천안시동남구,183142,89013,14851,36201,36603,87655,1358,94129


In [132]:
g5_chungnam_21.to_csv("temp2_1_governor_chungnam_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [133]:
g5_chungnam_22 = rename_columns_by_index(g5_chungnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_박해춘' -> '보수정당'
인덱스 5: '득표수_2_민주당_안희정' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [134]:
g5_chungnam_22 = add_sum_column(g5_chungnam_22, 6, 6, '그외정당')
g5_chungnam_22 = add_zero_column(g5_chungnam_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_자유선진당_박상돈

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [135]:
g5_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_자유선진당_박상돈,득표수_계,무효투표수,기권수,그외정당,무소속
0,충청남도,합계,1595587,901863,154723,367288,347265,869271,32587,693724,347265,0
1,충청남도,천안시서북구,221580,104685,16178,44605,42683,103466,1219,116895,42683,0
2,충청남도,천안시동남구,183142,89013,14851,36201,36603,87655,1358,94129,36603,0


In [136]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_chungnam_22 = g5_chungnam_22[selected_columns].copy()

g5_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청남도,합계,1595587,901863,154723,367288,347265,0,869271,32587,693724
1,충청남도,천안시서북구,221580,104685,16178,44605,42683,0,103466,1219,116895
2,충청남도,천안시동남구,183142,89013,14851,36201,36603,0,87655,1358,94129


In [137]:
g5_chungnam_22.to_csv("temp2_2_governor_chungnam_5.csv", index=False, encoding="utf-8-sig")

## Jeonbuk

In [138]:
g5_jeonbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_정운천,득표수_2_민주당_김완주,득표수_3_민주노동당_하연호,득표수_7_진보신당_염경석,득표수_8_평화민주당_김대식,득표수_계,무효투표수,기권수
0,전라북도,합계,1442805,856111,151064,569980,52331,35565,20990,829930,26181,586694
1,전라북도,전주시완산구,262926,140319,32656,86435,9616,7025,2685,138417,1902,122607
2,전라북도,전주시덕진구,210631,107769,22696,66511,7299,8168,1685,106359,1410,102862
3,전라북도,군산시,207328,113614,17270,81040,6297,4187,1766,110560,3054,93714
4,전라북도,익산시,234104,126680,23394,84786,7783,3928,3158,123049,3631,107424


In [139]:
show_column_info(g5_jeonbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_정운천
5: 득표수_2_민주당_김완주
6: 득표수_3_민주노동당_하연호
7: 득표수_7_진보신당_염경석
8: 득표수_8_평화민주당_김대식
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 16


### v2.1

In [140]:
g5_jeonbuk_21 = rename_columns_by_index(g5_jeonbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_정운천' -> '보수정당'
인덱스 5: '득표수_2_민주당_김완주' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [141]:
g5_jeonbuk_21 = add_sum_column(g5_jeonbuk_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_민주노동당_하연호
7: 득표수_7_진보신당_염경석
8: 득표수_8_평화민주당_김대식

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [142]:
g5_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_하연호,득표수_7_진보신당_염경석,득표수_8_평화민주당_김대식,득표수_계,무효투표수,기권수,그외정당
0,전라북도,합계,1442805,856111,151064,569980,52331,35565,20990,829930,26181,586694,108886
1,전라북도,전주시완산구,262926,140319,32656,86435,9616,7025,2685,138417,1902,122607,19326
2,전라북도,전주시덕진구,210631,107769,22696,66511,7299,8168,1685,106359,1410,102862,17152


In [143]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_jeonbuk_21 = g5_jeonbuk_21[selected_columns].copy()

g5_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라북도,합계,1442805,856111,151064,569980,108886,829930,26181,586694
1,전라북도,전주시완산구,262926,140319,32656,86435,19326,138417,1902,122607
2,전라북도,전주시덕진구,210631,107769,22696,66511,17152,106359,1410,102862


In [144]:
g5_jeonbuk_21.to_csv("temp2_1_governor_jeonbuk_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [145]:
g5_jeonbuk_22 = rename_columns_by_index(g5_jeonbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_정운천' -> '보수정당'
인덱스 5: '득표수_2_민주당_김완주' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [146]:
g5_jeonbuk_22 = add_sum_column(g5_jeonbuk_22, 6, 8, '그외정당')
g5_jeonbuk_22 = add_zero_column(g5_jeonbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_민주노동당_하연호
7: 득표수_7_진보신당_염경석
8: 득표수_8_평화민주당_김대식

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [147]:
g5_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_하연호,득표수_7_진보신당_염경석,득표수_8_평화민주당_김대식,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라북도,합계,1442805,856111,151064,569980,52331,35565,20990,829930,26181,586694,108886,0
1,전라북도,전주시완산구,262926,140319,32656,86435,9616,7025,2685,138417,1902,122607,19326,0
2,전라북도,전주시덕진구,210631,107769,22696,66511,7299,8168,1685,106359,1410,102862,17152,0


In [148]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_jeonbuk_22 = g5_jeonbuk_22[selected_columns].copy()

g5_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라북도,합계,1442805,856111,151064,569980,108886,0,829930,26181,586694
1,전라북도,전주시완산구,262926,140319,32656,86435,19326,0,138417,1902,122607
2,전라북도,전주시덕진구,210631,107769,22696,66511,17152,0,106359,1410,102862


In [149]:
g5_jeonbuk_22.to_csv("temp2_2_governor_jeonbuk_5.csv", index=False, encoding="utf-8-sig")

## Jeonnam

In [150]:
g5_jeonnam.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김대식,득표수_2_민주당_박준영,득표수_5_민주노동당_박웅두,득표수_7_평화민주당_김경재,득표수_계,무효투표수,기권수
0,전라남도,합계,1504902,967338,123548,629984,100581,68220,922333,45005,537564
1,전라남도,목포시,181872,91633,11266,60315,11789,5835,89205,2428,90239
2,전라남도,여수시,225147,130630,14500,89520,12618,9993,126631,3999,94517
3,전라남도,순천시,200321,122961,15447,79796,12483,10961,118687,4274,77360
4,전라남도,나주시,75246,52381,5423,34572,6582,3007,49584,2797,22865


In [151]:
show_column_info(g5_jeonnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_김대식
5: 득표수_2_민주당_박준영
6: 득표수_5_민주노동당_박웅두
7: 득표수_7_평화민주당_김경재
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 23


### v2.1

In [152]:
g5_jeonnam_21 = rename_columns_by_index(g5_jeonnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김대식' -> '보수정당'
인덱스 5: '득표수_2_민주당_박준영' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [153]:
g5_jeonnam_21 = add_sum_column(g5_jeonnam_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_5_민주노동당_박웅두
7: 득표수_7_평화민주당_김경재

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [154]:
g5_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_5_민주노동당_박웅두,득표수_7_평화민주당_김경재,득표수_계,무효투표수,기권수,그외정당
0,전라남도,합계,1504902,967338,123548,629984,100581,68220,922333,45005,537564,168801
1,전라남도,목포시,181872,91633,11266,60315,11789,5835,89205,2428,90239,17624
2,전라남도,여수시,225147,130630,14500,89520,12618,9993,126631,3999,94517,22611


In [155]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_jeonnam_21 = g5_jeonnam_21[selected_columns].copy()

g5_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라남도,합계,1504902,967338,123548,629984,168801,922333,45005,537564
1,전라남도,목포시,181872,91633,11266,60315,17624,89205,2428,90239
2,전라남도,여수시,225147,130630,14500,89520,22611,126631,3999,94517


In [156]:
g5_jeonnam_21.to_csv("temp2_1_governor_jeonnam_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [157]:
g5_jeonnam_22 = rename_columns_by_index(g5_jeonnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김대식' -> '보수정당'
인덱스 5: '득표수_2_민주당_박준영' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [158]:
g5_jeonnam_22 = add_sum_column(g5_jeonnam_22, 6, 7, '그외정당')
g5_jeonnam_22 = add_zero_column(g5_jeonnam_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_5_민주노동당_박웅두
7: 득표수_7_평화민주당_김경재

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [159]:
g5_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_5_민주노동당_박웅두,득표수_7_평화민주당_김경재,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라남도,합계,1504902,967338,123548,629984,100581,68220,922333,45005,537564,168801,0
1,전라남도,목포시,181872,91633,11266,60315,11789,5835,89205,2428,90239,17624,0
2,전라남도,여수시,225147,130630,14500,89520,12618,9993,126631,3999,94517,22611,0


In [160]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_jeonnam_22 = g5_jeonnam_22[selected_columns].copy()

g5_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라남도,합계,1504902,967338,123548,629984,168801,0,922333,45005,537564
1,전라남도,목포시,181872,91633,11266,60315,17624,0,89205,2428,90239
2,전라남도,여수시,225147,130630,14500,89520,22611,0,126631,3999,94517


In [161]:
g5_jeonnam_22.to_csv("temp2_2_governor_jeonnam_5.csv", index=False, encoding="utf-8-sig")

## Gyeongbuk

In [162]:
g5_gyeongbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김관용,득표수_2_민주당_홍의락,득표수_5_민주노동당_윤병태,득표수_7_국민참여당_유성찬,득표수_계,무효투표수,기권수
0,경상북도,합계,2122905,1260591,913812,143347,68015,87346,1212520,48071,862314
1,경상북도,포항시북구,199281,107009,81746,10702,4425,7961,104834,2175,92272
2,경상북도,포항시남구,197630,105544,77479,12764,5305,7620,103168,2376,92086
3,경상북도,울릉군,9072,7404,5825,648,288,326,7087,317,1668
4,경상북도,경주시,211517,124949,91802,12039,9766,7002,120609,4340,86568


In [163]:
show_column_info(g5_gyeongbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_김관용
5: 득표수_2_민주당_홍의락
6: 득표수_5_민주노동당_윤병태
7: 득표수_7_국민참여당_유성찬
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 25


### v2.1

In [164]:
g5_gyeongbuk_21 = rename_columns_by_index(g5_gyeongbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김관용' -> '보수정당'
인덱스 5: '득표수_2_민주당_홍의락' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [165]:
g5_gyeongbuk_21 = add_sum_column(g5_gyeongbuk_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_5_민주노동당_윤병태
7: 득표수_7_국민참여당_유성찬

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [166]:
g5_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_5_민주노동당_윤병태,득표수_7_국민참여당_유성찬,득표수_계,무효투표수,기권수,그외정당
0,경상북도,합계,2122905,1260591,913812,143347,68015,87346,1212520,48071,862314,155361
1,경상북도,포항시북구,199281,107009,81746,10702,4425,7961,104834,2175,92272,12386
2,경상북도,포항시남구,197630,105544,77479,12764,5305,7620,103168,2376,92086,12925


In [167]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_gyeongbuk_21 = g5_gyeongbuk_21[selected_columns].copy()

g5_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상북도,합계,2122905,1260591,913812,143347,155361,1212520,48071,862314
1,경상북도,포항시북구,199281,107009,81746,10702,12386,104834,2175,92272
2,경상북도,포항시남구,197630,105544,77479,12764,12925,103168,2376,92086


In [168]:
g5_gyeongbuk_21.to_csv("temp2_1_governor_gyeongbuk_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [169]:
g5_gyeongbuk_22 = rename_columns_by_index(g5_gyeongbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김관용' -> '보수정당'
인덱스 5: '득표수_2_민주당_홍의락' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [170]:
g5_gyeongbuk_22 = add_sum_column(g5_gyeongbuk_22, 6, 7, '그외정당')
g5_gyeongbuk_22 = add_zero_column(g5_gyeongbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_5_민주노동당_윤병태
7: 득표수_7_국민참여당_유성찬

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [171]:
g5_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_5_민주노동당_윤병태,득표수_7_국민참여당_유성찬,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상북도,합계,2122905,1260591,913812,143347,68015,87346,1212520,48071,862314,155361,0
1,경상북도,포항시북구,199281,107009,81746,10702,4425,7961,104834,2175,92272,12386,0
2,경상북도,포항시남구,197630,105544,77479,12764,5305,7620,103168,2376,92086,12925,0


In [172]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_gyeongbuk_22 = g5_gyeongbuk_22[selected_columns].copy()

g5_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상북도,합계,2122905,1260591,913812,143347,155361,0,1212520,48071,862314
1,경상북도,포항시북구,199281,107009,81746,10702,12386,0,104834,2175,92272
2,경상북도,포항시남구,197630,105544,77479,12764,12925,0,103168,2376,92086


In [173]:
g5_gyeongbuk_22.to_csv("temp2_2_governor_gyeongbuk_5.csv", index=False, encoding="utf-8-sig")

## Gyeongnam

In [174]:
g5_gyeongnam.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이달곤,득표수_7_무소속_김두관,득표수_계,무효투표수,기권수
0,경상남도,합계,2506393,1549690,705986,812336,1518322,31368,956703
1,경상남도,창원시,371540,224868,97364,125343,222707,2161,146672
2,경상남도,마산시,322188,191127,97429,91410,188839,2288,131061
3,경상남도,진주시,254687,164337,72465,89377,161842,2495,90350
4,경상남도,진해시,130168,77361,38717,37384,76101,1260,52807


In [175]:
show_column_info(g5_gyeongnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_이달곤
5: 득표수_7_무소속_김두관
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 21


### v2.1 (XXX)

In [176]:
g5_gyeongnam_21 = rename_columns_by_index(g5_gyeongnam, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_이달곤' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [177]:
g5_gyeongnam_21 = add_sum_column(g5_gyeongnam_21, 5, 5, '그외정당')
g5_gyeongnam_21 = add_zero_column(g5_gyeongnam_21, '진보정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_7_무소속_김두관

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 10


In [178]:
g5_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_7_무소속_김두관,득표수_계,무효투표수,기권수,그외정당,진보정당
0,경상남도,합계,2506393,1549690,705986,812336,1518322,31368,956703,812336,0
1,경상남도,창원시,371540,224868,97364,125343,222707,2161,146672,125343,0
2,경상남도,마산시,322188,191127,97429,91410,188839,2288,131061,91410,0


In [179]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_gyeongnam_21 = g5_gyeongnam_21[selected_columns].copy()

g5_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상남도,합계,2506393,1549690,705986,0,812336,1518322,31368,956703
1,경상남도,창원시,371540,224868,97364,0,125343,222707,2161,146672
2,경상남도,마산시,322188,191127,97429,0,91410,188839,2288,131061


In [180]:
g5_gyeongnam_21.to_csv("temp2_1_governor_gyeongnam_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, XX)

In [181]:
g5_gyeongnam_22 = rename_columns_by_index(g5_gyeongnam, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_이달곤' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [182]:
g5_gyeongnam_22 = add_sum_column(g5_gyeongnam_22, 5, 5, '무소속')
g5_gyeongnam_22 = add_zero_column(g5_gyeongnam_22, '진보정당')
g5_gyeongnam_22 = add_zero_column(g5_gyeongnam_22, '그외정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_7_무소속_김두관

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 10


In [183]:
g5_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_7_무소속_김두관,득표수_계,무효투표수,기권수,무소속,진보정당,그외정당
0,경상남도,합계,2506393,1549690,705986,812336,1518322,31368,956703,812336,0,0
1,경상남도,창원시,371540,224868,97364,125343,222707,2161,146672,125343,0,0
2,경상남도,마산시,322188,191127,97429,91410,188839,2288,131061,91410,0,0


In [184]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_gyeongnam_22 = g5_gyeongnam_22[selected_columns].copy()

g5_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상남도,합계,2506393,1549690,705986,0,0,812336,1518322,31368,956703
1,경상남도,창원시,371540,224868,97364,0,0,125343,222707,2161,146672
2,경상남도,마산시,322188,191127,97429,0,0,91410,188839,2288,131061


In [185]:
g5_gyeongnam_22.to_csv("temp2_2_governor_gyeongnam_5.csv", index=False, encoding="utf-8-sig")

## Jeju

In [186]:
g5_jeju.head()

,시도,구시군,선거인수,투표수,득표수_1_민주당_고희범,득표수_8_무소속_현명관,득표수_9_무소속_우근민,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,424098,276056,48186,108344,110603,267133,8923,148042
1,제주특별자치도,제주시,305765,195450,36447,74905,78514,189866,5584,110315
2,제주특별자치도,서귀포시,118333,80606,11739,33439,32089,77267,3339,37727


In [187]:
show_column_info(g5_jeju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_민주당_고희범
5: 득표수_8_무소속_현명관
6: 득표수_9_무소속_우근민
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 3


### v2.1 (XXX)

In [188]:
g5_jeju_21 = rename_columns_by_index(g5_jeju, 4, '진보정당')

인덱스 4: '득표수_1_민주당_고희범' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [189]:
g5_jeju_21 = add_sum_column(g5_jeju_21, 5, 6, '그외정당')
g5_jeju_21 = add_zero_column(g5_jeju_21, '보수정당')


=== 합산할 열들 (인덱스 5~6) ===
5: 득표수_8_무소속_현명관
6: 득표수_9_무소속_우근민

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [190]:
g5_jeju_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,득표수_8_무소속_현명관,득표수_9_무소속_우근민,득표수_계,무효투표수,기권수,그외정당,보수정당
0,제주특별자치도,합계,424098,276056,48186,108344,110603,267133,8923,148042,218947,0
1,제주특별자치도,제주시,305765,195450,36447,74905,78514,189866,5584,110315,153419,0
2,제주특별자치도,서귀포시,118333,80606,11739,33439,32089,77267,3339,37727,65528,0


In [191]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g5_jeju_21 = g5_jeju_21[selected_columns].copy()

g5_jeju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,424098,276056,0,48186,218947,267133,8923,148042
1,제주특별자치도,제주시,305765,195450,0,36447,153419,189866,5584,110315
2,제주특별자치도,서귀포시,118333,80606,0,11739,65528,77267,3339,37727


In [192]:
g5_jeju_21.to_csv("temp2_1_governor_jeju_5.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, XX)

In [193]:
g5_jeju_22 = rename_columns_by_index(g5_jeju, 4, '진보정당')

인덱스 4: '득표수_1_민주당_고희범' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [194]:
g5_jeju_22 = add_sum_column(g5_jeju_22, 5, 6, '무소속')
g5_jeju_22 = add_zero_column(g5_jeju_22, '보수정당')
g5_jeju_22 = add_zero_column(g5_jeju_22, '그외정당')


=== 합산할 열들 (인덱스 5~6) ===
5: 득표수_8_무소속_현명관
6: 득표수_9_무소속_우근민

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 11


In [195]:
g5_jeju_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,득표수_8_무소속_현명관,득표수_9_무소속_우근민,득표수_계,무효투표수,기권수,무소속,보수정당,그외정당
0,제주특별자치도,합계,424098,276056,48186,108344,110603,267133,8923,148042,218947,0,0
1,제주특별자치도,제주시,305765,195450,36447,74905,78514,189866,5584,110315,153419,0,0
2,제주특별자치도,서귀포시,118333,80606,11739,33439,32089,77267,3339,37727,65528,0,0


In [196]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g5_jeju_22 = g5_jeju_22[selected_columns].copy()

g5_jeju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,424098,276056,0,48186,0,218947,267133,8923,148042
1,제주특별자치도,제주시,305765,195450,0,36447,0,153419,189866,5584,110315
2,제주특별자치도,서귀포시,118333,80606,0,11739,0,65528,77267,3339,37727


In [197]:
g5_jeju_22.to_csv("temp2_2_governor_jeju_5.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [198]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp2_1_governor_gyeonggi_5.csv
Added: temp2_1_governor_daegu_5.csv
Added: temp2_2_governor_jeonbuk_5.csv
Added: temp2_1_governor_ulsan_5.csv
Added: temp2_1_governor_chungbuk_5.csv
Added: temp2_1_governor_incheon_5.csv
Added: temp2_1_governor_jeju_5.csv
Added: temp2_2_governor_chungbuk_5.csv
Added: temp2_1_governor_daejeon_5.csv
Added: temp2_2_governor_jeonnam_5.csv
Added: temp2_2_governor_jeju_5.csv
Added: temp2_2_governor_gwangju_5.csv
Added: temp2_2_governor_ulsan_5.csv
Added: temp2_1_governor_gyeongbuk_5.csv
Added: temp2_1_governor_gangwon_5.csv
Added: temp2_2_governor_incheon_5.csv
Added: temp2_2_governor_daegu_5.csv
Added: temp2_1_governor_jeonbuk_5.csv
Added: temp2_1_governor_jeonnam_5.csv
Added: temp2_2_governor_gyeonggi_5.csv
Added: temp2_1_governor_gwangju_5.csv
Added: temp2_2_governor_busan_5.csv
Added: temp2_2_governor_gyeongbuk_5.csv
Added: temp2_1_governor_seoul_5.csv
Added: temp2_2_governor_chungnam_5.csv
Added: temp2_2_governor_seoul_5.csv
Added: temp2_2_governor